# 4. 2つのDataFrameの比較・結合
<img src="04_DB01.png" alt="テーブル構造" title="テーブル">
<hr style="height:3px;background-color:#868686;">
<img src="table_structure.png" alt="テーブル構造" title="テーブル">
<hr style="height:3px;background-color:#868686;">
<img src="04_DB02.png" alt="テーブル構造" title="テーブル">
<hr style="height:3px;background-color:#868686;">
<img src="04_DBROMA.png" alt="テーブル構造" title="テーブル">

In [ ]:
#前準備1 データロード ZIPCODE_JP
import jaydebeapi #JDBC DB接続ライブラリ-
import pandas as pd
conn = jaydebeapi.connect(
    'com.ibm.db2.jcc.DB2Driver', #JDBC Driver名
    'jdbc:db2://localhost:50000/thinkjp',#接続先JDBC URL
     {'user': 'db2inst1', #DB接続ID
      'password':'thinkjapan'} , #DB接続pw  
     '/Applications/dsdriver/java/db2jcc4.jar'
     # JDBC Driver path (CLASSPATHが通っていれば不要)
    )

# DataFrame読み込み
df_jp = pd.read_sql("SELECT ZIPCODE, " +
                    "PREFECTURE_KANJI, "+
                    "CITY_KANJI, " +
                    "ADDRESS_KANJI " +
                    "FROM ZIPCODE_JP", conn)
                    
display(df_jp[0:10]) # 0から10行目までPrint

# 前準備2 データロード KEN_ALL_ROME.CSV
df_roma = pd.read_csv('KEN_ALL_ROME.CSV', 
                      encoding="shift-jis",#SJISかつMACの場合指定
                     names=['ZIPCODE', #指定したカラム名で読み込み
                    'PREFECTURE_KANJI','CITY_KANJI','ADDRESS_KANJI',
                    'PREFECTURE_R','CITY_R','ADDRESS_R'],
                      dtype='object'
                     )

display(df_roma[0:10]) # 0から10行目までPrint

print('df_jpの件数:{0}'.format(len(df_jp))) # 件数
print('df_romaの件数:{0}'.format(len(df_roma))) # 件数

In [ ]:
# チェック　実は違う！
display(df_jp.loc[[0], ['CITY_KANJI']])
display(df_roma.loc[[0], ['CITY_KANJI']])

# 差分の調査　カナ vs ローマ字

# 4-1: df_jp・df_roma間で合致&合致しないレコード
## 外部結合でindicatorで確認
    レコード数 df_jp:　124184
    レコード数 df_roma:　124117
    差分？
    文字内の全角スペースは削除して比較

In [ ]:
# 全角スペースの削除
# pyrthon上、文字列はデフォルトUTF-8
# '\u3000'はUTF-8の全角スペースを表している
df_jp = df_jp.apply(lambda x: x.str.replace('\u3000', '') 
                    if x.dtype == "object" else x )
df_roma = df_roma.apply(lambda x: x.str.replace('\u3000', '') 
                        if x.dtype == "object" else x )
                                                
# DBでもCSVでも同じDataFrame型なので、そのまま比較できる！
# 外部結合で比較
df_jp_check_outer = pd.merge(df_jp, df_roma, 
        left_on=['ZIPCODE','PREFECTURE_KANJI','CITY_KANJI','ADDRESS_KANJI'],
        right_on=['ZIPCODE','PREFECTURE_KANJI','CITY_KANJI','ADDRESS_KANJI'],
        how='outer',  #外部結合
        indicator=True) # indicatorをDataframeに含む _merge列に入る


print('df_jp,df_roma KEYが合致した件数:{0}'.format(
    len(df_jp_check_outer.loc[(df_jp_check_outer['_merge']=='both')])))

print('df_jpにしかないレコード件数:{0}'.format(
    len(df_jp_check_outer.loc[(df_jp_check_outer['_merge']=='left_only')])))

print('df_romaにしかないレコード件数:{0}'.format(
    len(df_jp_check_outer.loc[(df_jp_check_outer['_merge']=='right_only')])))

df_jp_check_outer.to_excel('jp_check.xlsx', index=False) #最後にSave

In [ ]:
# !をつけるとOSのコマンドが実行できます。以下はMACのコマンドです。
! ls -l jp_check.xlsx
! open -a Microsoft\ Excel jp_check.xlsx #EXCELのOPEN

In [ ]:
# 一番最初の合致しないレコードの郵便番号のデータをみてみます
df_jp_check_outer.loc[(df_jp_check_outer['ZIPCODE']=='0800029')]

In [ ]:
# df_jp 一番最初の合致しないレコードの郵便番号の'ADDRESS_KANJI'
df_jp.loc[(df_jp['ZIPCODE']=='0800029'),['ADDRESS_KANJI']]

In [ ]:
# df_roma 一番最初の合致しないレコードの郵便番号の'ADDRESS_KANJI'
df_roma.loc[(df_roma['ZIPCODE']=='0800029'),['ADDRESS_KANJI']]

# 以下は時間あれば・・・・
# 4-2: KEYの一致 

In [ ]:
# KEYが一致するレコードを結合する
# いわゆる内部結合
# KEYが一致するレコードのみ含まれる
df_match = pd.merge(df_jp, df_roma, 
         on=['ZIPCODE','PREFECTURE_KANJI','CITY_KANJI','ADDRESS_KANJI'])

print('df_jpの件数:{0}'.format(len(df_jp))) # 件数
print('df_romaの件数:{0}'.format(len(df_roma))) # 件数
print('KEYが一致した件数:{0}'.format(len(df_match))) # KEYが一致したレコード数

display(df_match) #KEYが完全一致するレコード

df_match.to_excel('match_jp_roma.xlsx', index=False) # 結果の保存


In [ ]:
# !をつけるとOSのコマンドが実行できます。以下はMACのコマンドです。
! ls -l match_jp_roma.xlsx
! open -a Microsoft\ Excel match_jp_roma.xlsx #EXCELのOPEN

# 4-3: df_jpレコードでdf_romeと合致しないレコード
## 左外部結合 後 合致しないものを抽出

In [ ]:
# df_jpと一致するdf_romaはKEYで結合される
# df_jpと一致しない場合はdf_jpの情報のみ
# いわゆる左外部結合、 SQLのLEFT OUTER JOIN
df_jp_check = pd.merge(df_jp, df_roma, 
         on=['ZIPCODE','PREFECTURE_KANJI','CITY_KANJI','ADDRESS_KANJI'],
        how='left',  #左外部結合
        indicator=True) # indicatorをDataframeに含む _merge列に入る

df_jp_check       
        

In [ ]:
# KEYがdf_romaと合致しないdf_jpのレコード
display(df_jp_check[['_merge']].drop_duplicates()) # _mergeのindicatorを確認
df_jp_check.loc[(df_jp_check['_merge']=='left_only')] # 合致しなかったレコード

# 4-4: df_romeレコードでdf_jpと合致しないレコード
## 右外部結合 後 合致しないものを抽出
(左外部結合でも可能)

In [ ]:
# df_romaと一致するdf_jpはKEYで結合される
# df_romaと一致しない場合はdf_romaの情報のみ
# いわゆる右外部結合、 SQLのRIGHT OUTER JOIN
df_jp_check = pd.merge(df_jp, df_roma, 
         on=['ZIPCODE','PREFECTURE_KANJI','CITY_KANJI','ADDRESS_KANJI'],
        how='right',  #右外部結合
        indicator=True) # indicatorをDataframeに含む _merge列に入る

# KEYがdf_jpと合致ないdf_romaのレコード
df_jp_check.loc[(df_jp_check['_merge']=='right_only')]

## merge -- ２つのDataFrameの比較に便利
how
- **inner**: 内部結合
- **left**: 左外部結合
- **right**: 右外部結合
- **outer**: 外部結合

keyの指定: 2つのDataFrameのKEYの名前が同じ場合
- **on**: 2つのDataFrameのKEYの名前が同じ場合

keyの指定: 2つのDataFrameのKEYの名前が同じ場合ではない場合
- **left_on**: 左側のDataFrameのKEYの名前
- **right_on**: 右側のDataFrameのKEYの名前



出力 _merge列 (indicator=Trueにして出力)
- **both** : 両方にマッチ
- **left_only** :左のDFのみ
- **right_only** :右のDFのみ